In [1]:
!mongoimport /d geo /c state ../datasets-master/states.json
!mongoimport /d geo /c airports ../datasets-master/airports.json
!mongoimport /d geo /c restaurants ../datasets-master/restaurants.json

2019-09-02T11:50:21.041+0900	connected to: mongodb://localhost/
2019-09-02T11:50:21.160+0900	50 document(s) imported successfully. 0 document(s) failed to import.
2019-09-02T11:50:22.156+0900	connected to: mongodb://localhost/
2019-09-02T11:50:22.268+0900	880 document(s) imported successfully. 0 document(s) failed to import.
2019-09-02T11:50:23.249+0900	connected to: mongodb://localhost/
2019-09-02T11:50:24.598+0900	25359 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
!mongoimport /d geo /c small_zips ../datasets-master/small_zips.json

2019-09-02T11:50:58.958+0900	connected to: mongodb://localhost/
2019-09-02T11:50:59.051+0900	200 document(s) imported successfully. 0 document(s) failed to import.


In [18]:
from pymongo import MongoClient, GEOSPHERE
from pprint import pprint

client = MongoClient()
db = client.geo
states = db.states
airports = db.airports
restaurants = db.restaurants
zips=db.small_zips

# Exercise 1

In [6]:
# create 2dsphere index
restaurants.create_index([("address.coord", GEOSPHERE)])

'address.coord_2dsphere'

In [19]:
# find international airports
area = airports.find({"type":"International"})

# TODO : find korean restaurants which are located 
#       at less 2km from international airports ($near)
for airport in area:
    cur = restaurants.find({'cusine':'Korean', 'address.coord':{'$near':{'$geometry':airport['loc'], '$maxDistance':2000}}})
    li = list(cur)
    if(len(li)>0):
        pprint(li)

# Exercise 2

In [12]:
# find the two coordinates (LAX, DTW)
lax = airports.find_one({'code': 'LAX'})
dtw = airports.find_one({'code': 'DTW'})

pprint(lax)
pprint(dtw)

{'_id': ObjectId('5d6c836e33b08211f1f1030d'),
 'code': 'LAX',
 'loc': {'coordinates': [-118.408084, 33.942537], 'type': 'Point'},
 'name': 'Los Angeles Intl',
 'type': 'International'}
{'_id': ObjectId('5d6c836e33b08211f1f1023f'),
 'code': 'DTW',
 'loc': {'coordinates': [-83.348836, 42.212052], 'type': 'Point'},
 'name': 'Detroit Metropolitan - Wayne County',
 'type': 'International'}


In [20]:
# make geojson format whose type is LineString (lax, dtw)
geojson = {
    'type': 'LineString',
    'coordinates': [lax['loc']['coordinates'],
                    dtw['loc']['coordinates']]
}

# TODO : find states that are intersect to the flight path
#       use $geoIntersects
result = states.find({
    'loc':{
        '$geoIntersects':{
            '$geometry': geojson
        }
    }
    
}, {'_id':0, 'name':1}).sort([('name', 1)])

pprint(list(result))

[]


In [7]:
client.close()